In [42]:
import math
import numpy as np
import pandas as pd
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA
import pmdarima as pm
import time
import xgboost as xgb
import matplotlib.pyplot as plt

from xgboost import plot_importance, plot_tree
from statsmodels.tsa.stattools import acf, pacf

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

In [2]:
full_df = pd.read_csv('../data/m4/daily-train.csv')

def get_ts(full_df, index):
    df = full_df.iloc[index:index+1, 1:].transpose()
    df.columns = ['y']
    df = df[df['y'].notna()]
    return df

# df = get_ts(full_df, 6)
df = get_ts(full_df, 1)
df.tail()

,y
V1003,2978.0
V1004,2991.9
V1005,2995.3
V1006,3000.5
V1007,2968.5


In [100]:
def generate_serial_lags(feature_count):
    return list(range(1, feature_count + 1))

def get_features_from_lags(df, lag_count=7, lags=None, keep_y=False, method="ratio"):
    """ Generates the feature (lags) that can than be used to forecast
        if the *lags* parameter is used, then the function will use the lags specified by this parameter and will
            ignore the lag_count parameter
        if lags is not used, the function will generate a sequence of features
            like [lag_1, lag_2, ..., lag_lag_count]
    """
    assert method in ['ratio', 'difference'], "method must be one of ['ratio', 'difference']"
    res = df[['y']].copy()
    
    if lags is None:
        lags = generate_serial_lags(lag_count)
    
    # The lags are computed for the *previous* value because when we forecast,
    # we cannot compute them for the current value
    res['previous'] = res['y'].shift(1)
    for lag in lags:
        if method == "ratio":
            res['lag_{}'.format(lag)] = res['previous'] / res['previous'].shift(lag) * 100
        else:
            res['lag_{}'.format(lag)] = res['previous'] - res['previous'].shift(lag)
    
#     res['day'] = np.arange(res.shape[0]) % 7
    if method == "ratio":
        res['lag_to_predict'] = res['y'] / res['previous'] * 100
    else:
        res['lag_to_predict'] = res['y'] - res['previous']
    
    # Ignore the first row, as it has no previous values, it cannot be predicted
    if keep_y:
        columns_to_drop = ['previous']
    else:
        columns_to_drop = ['previous', 'y']
    return res[1:].drop(columns_to_drop, axis=1)

get_features_from_lags(df, lags=[1, 2, 3, 4, 5, 6, 7])

,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_to_predict
V3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.004444
V4,100.444444,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
V5,100.000000,100.444444,NaN,NaN,NaN,NaN,NaN,0.986726
V6,98.672566,98.672566,99.111111,NaN,NaN,NaN,NaN,0.992825
V7,99.282511,97.964602,97.964602,98.400000,NaN,NaN,NaN,0.996387
...,...,...,...,...,...,...,...,...
V232,99.103139,97.100176,97.442681,98.660714,102.695167,101.376147,101.283226,1.004525
V233,100.452489,99.551570,97.539543,97.883598,99.107143,103.159851,101.834862,1.000000
V234,100.000000,100.452489,99.551570,97.539543,97.883598,99.107143,103.159851,1.000000
V235,100.000000,100.000000,100.452489,99.551570,97.539543,97.883598,99.107143,1.004505


In [101]:
def train_xgb(df, in_sample_validation=False):
    xgb_reg_params = {
    #         'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.1)),
        'learning_rate': 0.01,
        'max_depth':        5,
        'early_stopping_rounds': 10000,
        'eval_metric': 'rmse',
        'n_estimators': 8000,
    }

    if in_sample_validation:
        # Validate against in sample data
        train = df
        validation = df
    else:
        # Validate against out of sample
#         train = df[:-50]
#         validation = df[-50:]
        train, validation = train_test_split(df, test_size = 0.1)

    X_train = train.drop('lag_to_predict', axis=1)
    y_train = train[['lag_to_predict']]


    reg = xgb.XGBRegressor(**xgb_reg_params)
    eval_set = [(validation.drop('lag_to_predict', axis=1), validation[['lag_to_predict']])]
    reg.fit(X_train, y_train, eval_set=eval_set, verbose=False)
    
    return reg

# Use this method to convert from ratio prediction to actual y_hat which
# can be compared to 'y' from original df
def ratio_to_actual(df, predictions):
    df = df.copy()
    df['yhat'] = (df['y'] * predictions / 100).shift(1)
    return df

# Use this method to convert from lagged difference prediction to actual y_hat which
# can be compared to 'y' from original df
def lagged_diff_to_actual(df, predictions):
    df = df.copy()
    df['yhat'] = (df['y'] + predictions).shift(1)
    return df

def predict_xgb(df, model):
    return model.predict(df)

#The following ts can be used to showcase seasonality:61,71?,81,226, 246, 264
df = get_ts(full_df, 61)
lgs = get_features_from_lags(df, lags = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
train_df = lgs[:-10]
test_df = lgs[-10:].copy()
# test_df

model = train_xgb(train_df)
test_df['lag_hat'] = predict_xgb(test_df.drop('lag_to_predict', axis=1), model)
ratio_to_actual(df[-10:], test_df['lag_hat'])
# # plot_importance(model)

,y,yhat
V123,5862.5,NaN
V124,5895.0,57.738962
V125,5940.0,58.957878
V126,5972.5,58.933487
V127,5942.5,60.666248
V128,5875.0,59.920350
V129,5867.0,58.770822
V130,5715.0,58.040182
V131,5660.0,57.149724
V132,5659.0,56.438822


In [93]:
def get_preds(df):
    train_df = lgs[:-10]
    test_df = lgs[-10:].copy()

    model = train_xgb(train_df)
    test_df['lag_hat'] = predict_xgb(test_df.drop('lag_to_predict', axis=1), model)

    return test_df


index = 61


df = get_ts(full_df, index)

all_results = []
for method in ['ratio', 'difference']:
    lgs = get_features_from_lags(df, lags = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], method=method)
    preds = get_preds(lgs)
    
    # First row does not have a prediction so drop it
#     preds = preds.iloc[1:,:]
    
    if method == 'ratio':
        res_df = ratio_to_actual(df[-preds.shape[0]:], preds['lag_hat'])
    else:
        res_df = lagged_diff_to_actual(df[-preds.shape[0]:], preds['lag_hat'])

    res_df['ts_index'] = index
    res_df['method'] = method
    all_results.extend(res_df.to_dict('records'))

pd.DataFrame(all_results)

,y,yhat,ts_index,method
0,5862.5,NaN,61,ratio
1,5895.0,5877.009824,61,ratio
2,5940.0,5933.935944,61,ratio
3,5972.5,5935.882026,61,ratio
4,5942.5,6015.452178,61,ratio
5,5875.0,5992.634262,61,ratio
6,5867.0,5943.565413,61,ratio
7,5715.0,5842.711205,61,ratio
8,5660.0,5732.045653,61,ratio
9,5659.0,5678.264794,61,ratio


In [103]:
def get_preds(df):
    train_df = lgs[:-10]
    test_df = lgs[-10:].copy()

    model = train_xgb(train_df)
    test_df['lag_hat'] = predict_xgb(test_df.drop('lag_to_predict', axis=1), model)

    return test_df


index = 61

all_results = []

for index in tqdm(range(20)):
    df = get_ts(full_df, index)
    for method in ['ratio', 'difference']:
        lgs = get_features_from_lags(df, lags = [1, 2, 3, 4, 5, 6, 7], method=method)
        preds = get_preds(lgs)

        if method == 'ratio':
            res_df_ratio = ratio_to_actual(df[-preds.shape[0]:], preds['lag_hat']).iloc[1:, :]
        else:
            res_df_diffs = lagged_diff_to_actual(df[-preds.shape[0]:], preds['lag_hat']).iloc[1:, :]

        # First row does not have a prediction that we can compare so drop it
        # (that is because we don't have previous values to predict on)
    res_df_all = res_df_ratio.copy()
    res_df_all.rename(columns={'yhat':'yhat_ratio'}, inplace=True)
    res_df_all['yhat_diffs'] = res_df_diffs['yhat']
    res_df_all['ts_index'] = index
    res_df_all['method'] = method
    all_results.extend(res_df_all.to_dict('records'))

res_df = pd.DataFrame(all_results)
res_df['mse_ratio'] = (res_df['y'] - res_df['yhat_ratio'])**2
res_df['mse_diffs'] = (res_df['y'] - res_df['yhat_diffs'])**2

print("Ratio performed better in {} ({:2f}%) of the cases".format(
        res_df[res_df['mse_ratio'] < res_df['mse_diffs']].shape[0],
        res_df[res_df['mse_ratio'] < res_df['mse_diffs']].shape[0] / res_df.shape[0] * 100
    )
)

print("MSE for Ratio is {:2f}".format(mean_squared_error(res_df['y'], res_df['yhat_ratio'])))
print("MSE for Diffs is {:2f}".format(mean_squared_error(res_df['y'], res_df['yhat_diffs'])))

res_df

  0%|          | 0/20 [00:00<?, ?it/s]

Ratio performed better in 0 (0.000000%) of the cases
MSE for Ratio is 23906353.625709
MSE for Diffs is 62923.400069


,y,yhat_ratio,yhat_diffs,ts_index,method,mse_ratio,mse_diffs
0,2002.9,20.061120,2009.189299,0,difference,3.931650e+06,39.555287
1,2012.6,19.998492,2000.384748,0,difference,3.970461e+06,149.212373
2,2013.8,20.247834,2012.681047,0,difference,3.974250e+06,1.252055
3,2003.4,20.107481,2014.133639,0,difference,3.933449e+06,115.210996
4,2015.6,20.013702,2003.959926,0,difference,3.982365e+06,135.491326
...,...,...,...,...,...,...,...
175,10907.1,108.269564,10848.094902,19,difference,1.166147e+08,3481.601574
176,10866.0,108.947472,10907.594902,19,difference,1.157142e+08,1730.135884
177,10865.2,109.410889,10869.574248,19,difference,1.156870e+08,19.134042
178,10855.3,108.485699,10868.774248,19,difference,1.154940e+08,181.555348


In [99]:
def get_preds(df):
    train_df = lgs[:-10]
    test_df = lgs[-10:].copy()

    model = train_xgb(train_df)
    test_df['lag_hat'] = predict_xgb(test_df.drop('lag_to_predict', axis=1), model)

    return test_df


index = 61

all_results = []

for index in tqdm(range(2000)):
    df = get_ts(full_df, index)
    for method in ['ratio', 'difference']:
        lgs = get_features_from_lags(df, lags = [1, 2, 3, 4, 5, 6, 7], method=method)
        preds = get_preds(lgs)

        if method == 'ratio':
            res_df_ratio = ratio_to_actual(df[-preds.shape[0]:], preds['lag_hat']).iloc[1:, :]
        else:
            res_df_diffs = lagged_diff_to_actual(df[-preds.shape[0]:], preds['lag_hat']).iloc[1:, :]

        # First row does not have a prediction that we can compare so drop it
        # (that is because we don't have previous values to predict on)
    res_df_all = res_df_ratio.copy()
    res_df_all.rename(columns={'yhat':'yhat_ratio'}, inplace=True)
    res_df_all['yhat_diffs'] = res_df_diffs['yhat']
    res_df_all['ts_index'] = index
    res_df_all['method'] = method
    all_results.extend(res_df_all.to_dict('records'))

res_df = pd.DataFrame(all_results)
res_df['mse_ratio'] = (res_df['y'] - res_df['yhat_ratio'])**2
res_df['mse_diffs'] = (res_df['y'] - res_df['yhat_diffs'])**2

print("Ratio performed better in {} ({:2f}%) of the cases".format(
        res_df[res_df['mse_ratio'] < res_df['mse_diffs']].shape[0],
        res_df[res_df['mse_ratio'] < res_df['mse_diffs']].shape[0] / res_df.shape[0] * 100
    )
)

print("MSE for Ratio is {:2f}".format(mean_squared_error(res_df['y'], res_df['yhat_ratio'])))
print("MSE for Diffs is {:2f}".format(mean_squared_error(res_df['y'], res_df['yhat_diffs'])))

res_df

  0%|          | 0/2000 [00:00<?, ?it/s]

Ratio performed better in 8206 (45.588889%) of the cases
MSE for Ratio is 17774.771121
MSE for Diffs is 13340.408003


,y,yhat_ratio,yhat_diffs,ts_index,method,mse_ratio,mse_diffs
0,2002.9,2003.131027,2008.705748,0,difference,0.053374,33.706711
1,2012.6,1996.745970,2003.396522,0,difference,251.350270,84.704003
2,2013.8,2003.297198,2013.118446,0,difference,110.308848,0.464516
3,2003.4,2007.363653,2014.296522,0,difference,15.710545,118.734196
4,2015.6,1998.172145,2003.918446,0,difference,303.730129,136.458705
...,...,...,...,...,...,...,...
17995,1110.0,1095.062373,1103.665505,1999,difference,223.132701,40.125831
17996,1110.0,1101.644111,1113.696604,1999,difference,69.820886,13.664881
17997,1110.0,1102.369965,1108.651150,1999,difference,58.217438,1.819396
17998,1115.0,1104.780017,1114.696742,1999,difference,104.448050,0.091966
